In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
df = pd.read_csv('amazon_products_cleaned.csv')
data = pd.read_csv('amazon_products.csv')

In [3]:
df['category_id'].nunique()

19

In [4]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')
embeddings = model.encode(df['category_id'].tolist(), show_progress_bar=True, device='cuda')

Batches:   0%|          | 0/44574 [00:00<?, ?it/s]

In [5]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=10, random_state=42)
df['embeddings'] = kmeans.fit_predict(embeddings)
df['embeddings'] = df['embeddings'].astype(str)

  File "C:\Users\JAIVAL CHAUHAN\AppData\Roaming\Python\Python312\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "c:\Program Files\Python312\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Program Files\Python312\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "c:\Program Files\Python312\Lib\subprocess.py", line 1538, in _execute_child
    hp, ht, pid, tid = _winapi.CreateProcess(executable, args,
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


In [7]:
df.head()

,asin,title,imgUrl,productURL,stars,reviews,price,listPrice,category_id,isBestSeller,boughtInLastMonth,embeddings
0,B014TMV5YE,"Sion Softside Expandable Roller Luggage, Black...",https://m.media-amazon.com/images/I/815dLQKYIY...,https://www.amazon.com/dp/B014TMV5YE,4.5,0,139.99,0.00,104,False,2000,9
1,B07GDLCQXV,Luggage Sets Expandable PC+ABS Durable Suitcas...,https://m.media-amazon.com/images/I/81bQlm7vf6...,https://www.amazon.com/dp/B07GDLCQXV,4.5,0,169.99,209.99,104,False,1000,9
2,B07XSCCZYG,Platinum Elite Softside Expandable Checked Lug...,https://m.media-amazon.com/images/I/71EA35zvJB...,https://www.amazon.com/dp/B07XSCCZYG,4.6,0,365.49,429.99,104,False,300,9
3,B08MVFKGJM,Freeform Hardside Expandable with Double Spinn...,https://m.media-amazon.com/images/I/91k6NYLQyI...,https://www.amazon.com/dp/B08MVFKGJM,4.6,0,291.59,354.37,104,False,400,9
4,B01DJLKZBA,Winfield 2 Hardside Expandable Luggage with Sp...,https://m.media-amazon.com/images/I/61NJoaZcP9...,https://www.amazon.com/dp/B01DJLKZBA,4.5,0,174.99,309.99,104,False,400,9


In [14]:
def recommend_similar_by_text(product_id, df, top_n=10):
    cluster_id = df[df['asin'] == product_id]['embeddings'].values[0]
    similar_products = df[df['embeddings'] == cluster_id]
    return similar_products[similar_products['asin'] != product_id][['asin', 'title', 'price']].head(top_n)

In [15]:
recommend_similar_by_text('B07XSCCZYG', df)

,asin,title,price
0,B014TMV5YE,"Sion Softside Expandable Roller Luggage, Black...",139.99
1,B07GDLCQXV,Luggage Sets Expandable PC+ABS Durable Suitcas...,169.99
3,B08MVFKGJM,Freeform Hardside Expandable with Double Spinn...,291.59
4,B01DJLKZBA,Winfield 2 Hardside Expandable Luggage with Sp...,174.99
5,B07XSCD2R4,Maxlite 5 Softside Expandable Luggage with 4 S...,144.49
6,B07MXF4G8K,"Hard Shell Carry on Luggage Airline Approved, ...",169.99
7,B07H515VCZ,"Maxporter II 30"" Hardside Spinner Trunk Luggag...",299.99
8,B08BXBCNMQ,Omni 2 Hardside Expandable Luggage with Spinne...,112.63
9,B0B9K44XTS,Luggage Sets Expandable Lightweight Suitcases ...,209.99
10,B07QZLHTTY,Crew Versapack Softside Expandable 8 Spinner W...,271.99


In [ ]:
df.to_csv('amazon_products_cleaned.csv', index=False)